# `014` Attention mechanisms

Requirements: 010 Embeddings, 013 LSTM

⚠️ WIP

There is a fundamental problem with LSTM, GRU and other RNNs, which is that they are not feedforward, so that the individual timesteps of processing a sequence cannot be parallelized. This is a problem because it makes the training slow. Plus, the RNNs are not very good at capturing very long-term dependencies.

There is a different kind of layer called `Attention` that can be used to solve this problem. The idea is to have a layer that can look at the entire sequence at once and decide which parts of the sequence are important for the current timestep. This way, the layer can be parallelized and can capture long-term dependencies. The underlying idea is converting every element in the sequence into a linear combination of all the elements in the sequence, with the weights of the linear combination being learned. Let's see an implementation in code:

In [66]:
from json import loads
from matplotlib import pyplot as plt
from string import ascii_letters, digits
from time import time
from unicodedata import category, normalize
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vocabulary = ascii_letters + digits + ' .,;\'!'
c2i = {c: i for i, c in enumerate(vocabulary)}
i2c = {i: c for i, c in enumerate(vocabulary)}

def vectorize_sentence(s):
	return [c2i[c] for c in normalize('NFD', s) if category(c) != 'Mn' and c in vocabulary]

In [65]:
input = torch.tensor([vectorize_sentence('Hello, World!')])

embedding_channels = 8
embeddings = torch.randn(len(vocabulary), embedding_channels)
input = embeddings[input]

head_size = 16
W_q = torch.randn(embedding_channels, head_size)
W_k = torch.randn(embedding_channels, head_size)
W_v = torch.randn(embedding_channels, head_size)

q = input @ W_q
k = input @ W_k
attention_weights = q @ k.transpose(-2, -1).softmax(dim=-1)

v = input @ W_v
output = attention_weights @ v